In [ ]:
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds
from holoviews.operation.datashader import rasterize, datashade
from colorcet import bmy
from holoviews.util import Dynamic
hv.extension('bokeh', width=100)


In [ ]:
df = pd.read_hdf('./data/rgi62_era5_itmix_df.h5', 'df')
df['latdeg'] = df.CenLat
df.tail()

In [ ]:
data = gv.Points(df, [('CenLon', 'Longitude'), ('CenLat', 'Latitude')],
                     [('era5_avg_pcp', 'Annual Precipitation (mm/yr)'),
                      ('Area', 'Area'),  ('latdeg', 'Latitude (deg)'),
                      ('era5_avg_temp_at_zmed', 'Annual Temperature at avg. altitude'), 
                      ('Zmed', 'Elevation')])
total_area = df.Area.sum()
print(data, len(data), total_area)

In [ ]:
data = gv.Dataset(gv.operation.project_points(data))

In [ ]:
geo_kw    = dict(aggregator=ds.sum('Area'), x_sampling=1000, y_sampling=1000)
elev_kw   = dict(cmap='#7d3c98')
temp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('era5_avg_temp_at_zmed'))
prcp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('era5_avg_pcp'))

size_opts = dict(min_height=400, min_width=600, responsive=True)
geo_opts  = dict(size_opts, cmap=bmy, global_extent=False, logz=True, colorbar=True)
elev_opts = dict(size_opts, show_grid=True)
temp_opts = dict(size_opts, fill_color='#f1948a', default_tools=[], toolbar=None, alpha=1.0)
prcp_opts = dict(size_opts, fill_color='#85c1e9', default_tools=[], toolbar=None, alpha=1.0)

In [ ]:
geo_bg = gv.tile_sources.EsriImagery.options(alpha=0.6, bgcolor="black")
geopoints = data.to(gv.Points, ['CenLon', 'CenLat'], ['Area'], []).options(**geo_opts).redim.range(area_km2=(0, 3000))

(geo_bg*rasterize(geopoints, **geo_kw).options(**geo_opts) + 
 datashade(data.to(hv.Scatter, 'Zmed','latdeg', []), **elev_kw).options(**elev_opts) + 
 data.hist('era5_avg_temp_at_zmed', **temp_kw).options(**temp_opts) +
 data.hist('era5_avg_pcp',              **prcp_kw).options(**prcp_opts)).cols(2)



In [ ]:
def geo(data):   return gv.Points(data, crs=ccrs.GOOGLE_MERCATOR).options(alpha=1)
def elev(data):  return data.to(hv.Scatter, 'Zmed', 'latdeg', [])
def temp(data):  return data.hist('era5_avg_temp_at_zmed', **temp_kw).options(**temp_opts)
def prcp(data):  return data.hist('era5_avg_pcp',              **prcp_kw).options(**prcp_opts)
def count(data): return hv.Div('<p style="font-size:20px">Glaciers selected: {}'.format(len(data)) + "<br>" +
                               'Area: {:.0f} km² ({:.1f}%)</font>'.format(np.sum(data['Area']), np.sum(data['Area']) / total_area * 100)).options(height=40)


In [ ]:
static_geo  = rasterize(geo(data),   **geo_kw).options(alpha=0.1, tools=['hover', 'box_select'], active_tools=['box_select'], **geo_opts)
static_elev = datashade(elev(data), **elev_kw).options(alpha=0.1, tools=[         'box_select'], active_tools=['box_select'], **elev_opts)
static_temp = temp(data).options(alpha=0.1)
static_prcp = prcp(data).options(alpha=0.1)

In [ ]:
def combine_selections(**kwargs):
    """
    Combines selections on all available plots into a single selection by index.
    """
    if all(not v for v in kwargs.values()):
        return slice(None)
    selection = {}
    for key, bounds in kwargs.items():
        if bounds is None:
            continue
        elif len(bounds) == 2:
            selection[key] = bounds
        else:
            xbound, ybound = key.split('__')
            selection[xbound] = bounds[0], bounds[2]
            selection[ybound] = bounds[1], bounds[3]
    return sorted(set(data.select(**selection).data.index))

def select_data(**kwargs):
    return data.iloc[combine_selections(**kwargs)] if kwargs else data



In [ ]:
select_data(CenLon__CenLat=(3000000, 9000000, 5000000, 11000000)).dframe()

In [ ]:
select_data(CenLon__CenLat=(3000000, 9000000, 5000000, 11000000), era5_avg_temp_at_zmed=(-5,-2)).dframe()

In [ ]:
from holoviews.streams import Stream, BoundsXY, BoundsX

geo_bounds  = BoundsXY(source=static_geo,  rename={'bounds':  'CenLon__CenLat'})
elev_bounds = BoundsXY(source=static_elev, rename={'bounds':  'Zmed__latdeg'})
temp_bounds = BoundsX( source=static_temp, rename={'boundsx': 'era5_avg_temp_at_zmed'})
prcp_bounds = BoundsX( source=static_prcp, rename={'boundsx': 'era5_avg_pcp'})

selections  = [geo_bounds, elev_bounds, temp_bounds, prcp_bounds]



In [ ]:
dyn_data  = hv.DynamicMap(select_data, streams=selections)

dyn_geo   = rasterize(dyn_data.apply(geo),  **geo_kw).options( **geo_opts)
dyn_elev  = datashade(dyn_data.apply(elev), **elev_kw).options(**elev_opts)
dyn_temp  =           dyn_data.apply(temp)
dyn_prcp  =           dyn_data.apply(prcp)
dyn_count =           dyn_data.apply(count)



In [ ]:
geomap = geo_bg * static_geo  * dyn_geo
elevation       = static_elev * dyn_elev
temperature     = static_temp * dyn_temp
precipitation   = static_prcp * dyn_prcp

In [ ]:
def clear_selections(arg=None):
    geo_bounds.update(bounds=None)
    elev_bounds.update(bounds=None)
    temp_bounds.update(boundsx=None)
    prcp_bounds.update(boundsx=None)
    Stream.trigger(selections)

In [ ]:
import panel as pn
pn.extension()

clear_button = pn.widgets.Button(name='Clear selection')
clear_button.param.watch(clear_selections, 'clicks');

In [ ]:
title       = '<div style="font-size:35px">World glaciers explorer</div>'
instruction = 'Box-select on each plot to subselect; clear selection to reset.<br>' + \
              'See the <a href="https://github.com/panel-demos/glaciers">Jupyter notebook</a> source code for how to build apps like this!'
oggm_logo   = '<a href="https://oggm.org"><img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logos/oggm_s_alpha.png" width=170></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=140></a>'

In [ ]:
header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Pane(title, width=400), pn.Pane(instruction, width=500)),
                pn.layout.HSpacer(), pn.Column(pn.Pane(dyn_count), pn.layout.Spacer(height=20), clear_button), 
                pn.Pane(pn_logo, width=140))

pn.Column(header,  pn.Row(geomap, elevation), pn.Row(temperature, precipitation), width_policy='max', height_policy='max').servable()